### ILQR example
1. The example shows how to compute the ilqr solution to a 1 dof actuated pendulum
2. At this point we are only penalizing the control inputs and the deviation from the final state in the overall cost

#### Discrete dynamics
1. In the following the symbolic dynamics of the pendulum is defined.
2. For convenience, lambda functions are used for:
    - The discrete dynamics $x_{k+1} = f(u_k, x_k)$
    - The linearized system with $A = \frac{\partial f}{\partial x}$ and $B = \frac{\partial f}{\partial u}$

In [24]:
import numpy as np
import sympy as sp
from sympy.matrices import Matrix

In [25]:
def symbolic_dynamics_pendulum():
    """Symbolic dynamics for a pendulum

    Returns:
        sympy.Function: f(x.u)
        sympy.Function: df/dx
        sympy.Function: df/du
    """
    m, g, L, theta, theta_dot, u, dt = sp.symbols('m g L theta theta_dot u dt')

    # inputs and states
    inputs = Matrix([u])
    states = Matrix([theta, theta_dot])

    # dynamics for a pendulum of mass m and center of mass L
    # ----------------
    #        |\
    #        | \
    # \theta |->\
    #        |  []
    f = Matrix([theta_dot, (u-m*g*L*sp.sin(theta))/(m*L*L)])

    # discretize the system using euler integration
    f_disc = states + f*dt

    # first derivatives wrt to (x, u)
    f_x = f_disc.jacobian(states) # df/dx
    f_u = f_disc.jacobian(inputs) # df/du

    # second derivatives wrt to (x, u)
    # f_xx = f_x.(states) # d2f/dx2
    # f_uu = f_u.jacobian(states) # d2f/du2

    # define parameters
    parameters = Matrix([m,g,L])

    # create lambdas
    f_func = sp.lambdify((states, inputs, dt, parameters), f_disc)
    f_x_func = sp.lambdify((states, inputs, dt, parameters), f_x)
    f_u_func = sp.lambdify((states, inputs, dt, parameters), f_u)
    # f_xx_func = sp.lambdify((states, inputs, dt, parameters), f_xx)
    # f_uu_func = sp.lambdify((states, inputs, dt, parameters), f_uu)

    # return (f_func, f_x_func, f_u_func, f_xx_func, f_uu_func)
    return (f_func, f_x_func, f_u_func)


#### Iterative LQR algorithm
1. 

In [26]:
class ilqr:
    def __init__(self, init_state, target_state, initial_input_guess, dt, start_time, end_time, f_discrete, f_x, f_u, Q_k, R_k, Q_T, parameters, n_iterations):
        """Initialization of ILQR

        Args:
            init_state (ndarray): Initial state
            target_state (ndarray): Target state
            initial_input_guess (ndarray): Initial guess for ilqr
            dt (double): Sampling time for discrete system
            start_time (double): Starting time, defaults to 0 for a single trajectory
            end_time (double): Ending time, defaults to final time for a single trajectory
            f_discrete (sympy.Matrix): Symbolic representation of the x_{k_+1} = f(x_k, u_k)
            f_x (sympy.Matrix): df_dx term of the linearized system
            f_u (sympy.Matrix): df_du term of the linearized system
            Q_k (ndarray): Weights for states in the running cost
            R_k (ndarray): Weights for inputs in the running cost
            Q_T (ndarray): Weights for states in the terminal cost
            parameters (ndarray): Parameters of the system
            n_iterations (double): Maximum interations for ilqr
        """
        # states
        self.init_state_ = init_state
        self.target_state_ = target_state
        self.inputs_ = initial_input_guess
        self.n_states_ = np.shape(init_state)[0] # The dimensions of the state vector
        self.n_inputs_ = np.shape(initial_input_guess)[1] # The dimension of the control vector

        # timing
        self.dt_ = dt
        self.start_time_ = start_time
        self.end_time_ = end_time
        self.time_span_ = np.arange(start_time, end_time, dt).flatten()
        self.n_timesteps_ = np.shape(self.time_span_)[0]

        # dynamics
        self.f_ = f_discrete
        self.f_x_ = f_x
        self.f_u_ = f_u

        # weighting for loss function, i.e. L = x_T^T Q_T x_T + sum of (x_k^T Q_k x_k + u_k^T R_k u_k)
        self.Q_k_ = Q_k # Weight for state vector
        self.R_k_ = R_k # Weight for control vector
        self.Q_T_ = Q_T # Weight for terminal state
        self.parameters_ = parameters

        # max iterations to run
        self.n_iterations_ = n_iterations

        # costs
        self.expected_cost_reduction_ = 0
        self.expected_cost_reduction_grad_ = 0
        self.expected_cost_reduction_hess_ = 0

    def rollout(self):
        """Rollout of the simulated system given an initial state

        Returns:
            ndarray: States trajectory from the rollout
            ndarray: Inputs trajectory from the rollout
        """
        # we store states and inputs as:
        # state = [., x_1, x_2, ..., x_N]
        # input =    [u_0, u_1, ..., u_{N-1}]
        # the first value in state is understood as self.init_state_
        states = np.zeros((self.n_timesteps_+1, self.n_states_)) # including initial state, x_0 to x_N
        inputs = np.zeros((self.n_timesteps_, self.n_inputs_)) # u_0 to u_{N-1}

        # TODO: store first state, x0 into states
        states[0,:] = self.init_state_

        print(self.n_timesteps_)

        # integrate dynamics
        current_state = self.init_state_
        for i in range(0, self.n_timesteps_): # 0 to N-1
            current_input = self.inputs_[i,:] # u_k
            print("i = {}, current_input = {}, current_state = {}".format(i, current_input, current_state))
            next_state = self.f_(current_state, current_input, self.dt_, self.parameters_).flatten() # x_{k+1} = f(x_k, u_k) 
            # print("i = {}, next_state = {}".format(i, next_state))
            # store both u_k and x_{k+1}
            states[i+1,:] = next_state
            inputs[i,:] = current_input
            # update current state
            current_state = next_state

        # store trajectories
        self.states_ = states
        self.inputs_ = inputs

        return states, inputs

    def compute_cost(self, states, inputs):
        """Computes the cost from all the terms, i.e. dynamics and cost as well as their derivatives:
        f_x, f_u, f_xx, f_ux, f_uu, 
        l_x, l_u, l_xx, l_ux, l_uu

        Args:
            states (ndarray): State trajectory
            inputs (ndarray): Input trajectory

        Returns:
            double: Total cost, i.e. terminal cost + running cost
        """
        # dynamics first derivatives

        # dynamics second derivatives

        # cost first derivatives

        # cost second derivatives

        # accumulate cost to go
        total_cost = 0
        for i in range(0, self.n_timesteps_):
            current_x = states[i,:]
            current_u = inputs[i,:].flatten()
            current_cost = current_u.T @ self.R_k_ @ current_u
            total_cost = total_cost + current_cost
        # add terminal cost
        terminal_diff = (states[-1,:] - self.target_state_).flatten()
        terminal_cost = terminal_diff.T @ self.Q_T_ @ terminal_diff
        total_cost = total_cost + terminal_cost

        return total_cost

    def backward_pass(self):
        """Backward pass of iLQR

        Returns:
            ndarray: feedforward gain, k
            ndarray: feedback gain, K
            double: expected cost reduction
        """
        # starting from the last state
        V_xx = self.Q_T_ # since V_N = x_T^T Q_T x_T, V_xx(N) = Q_T_
        end_difference = (self.states_[-1, :] - self.target_state_).flatten()
        end_difference = end_difference.flatten()
        V_x = self.Q_T_ @ end_difference # V_x(N)

        # initialize control modifications to be stored
        k_trj = np.zeros((self.n_timesteps_, self.n_inputs_)) # (8b)
        K_trj = np.zeros((self.n_timesteps_, self.n_inputs_, self.n_states_)) # (8b)

        # initialize cost reduction
        expected_cost_reduction = 0
        expected_cost_reduction_grad = 0
        expected_cost_reduction_hess = 0

        # looping backwards from N-1 to 1 using initial value of V_{N}
        for i in reversed(range(0, self.n_timesteps_)):
            # current variables
            current_x = self.states_[i,:]
            current_u = self.inputs_[i,:]

            # updates to partial derivatives of cost function
            l_xx = self.Q_k_
            l_uu = self.R_k_

            # l_ux = np.zeros((self.n_inputs_, self.n_states_))
            l_x = self.Q_k_ @ np.zeros(self.n_states_).flatten()
            l_u = self.R_k_ @ (current_u).flatten()

            # get jacobian of discrete dynamics
            f_x = self.f_x_(current_x, current_u, self.dt_, self.parameters_) # V'_x
            f_u = self.f_u_(current_x, current_u, self.dt_, self.parameters_) # V'_u
    
            # all the Q vector/matrices
            Q_x = l_x + f_x.T @ V_x # (5a)
            Q_u = l_u + f_u.T @ V_x # (5b)
            Q_ux = f_u.T @ V_xx @ f_x # (5c)
            Q_uu = l_uu + f_u.T @ V_xx @ f_u # (5d)
            Q_xx = l_xx + f_x.T @ V_xx @ f_x # (5e)

            # compute and store gains
            kSingValThreshold = 1e-4
            (_,s,_) = np.linalg.svd(Q_uu)
            if (np.min(s) < kSingValThreshold):
                print("Q_uu is non-singular")
            Q_uu_inv = np.linalg.inv(Q_uu) # TODO: this can be singular, try using (9)
            k = -Q_uu_inv @ Q_u # (6)
            K = -Q_uu_inv @ Q_ux # (6)

            k_trj[i,:] = k 
            K_trj[i,:,:] = K 

            # update the expected reduction (11a), delta V
            # similar to equation of delta J(\alpha)
            current_cost_reduction_grad = -Q_u.T @ k
            current_cost_reduction_hess = (0.5 * k.T @ (Q_uu) @ (k))
            current_cost_reduction = current_cost_reduction_grad + current_cost_reduction_hess

            expected_cost_reduction_grad += current_cost_reduction_grad
            expected_cost_reduction_hess += current_cost_reduction_hess
            expected_cost_reduction += current_cost_reduction

            # update hessian and gradient of value function for the next iteration
            V_x = Q_x + K.T @ Q_uu @ k + K.T @ Q_u + Q_ux.T @ k # (11b)
            V_xx = Q_xx + K.T @ Q_uu @ K + K.T @ Q_ux + Q_ux.T @ K # (11c)

        # store values
        self.expected_cost_reduction_grad_ = expected_cost_reduction_grad
        self.expected_cost_reduction_hess_ = expected_cost_reduction_hess
        self.expected_cost_reduction_ = expected_cost_reduction

        # store gains
        self.k_feedforward_ = k_trj
        self.K_feedback_ = K_trj

        return (k_trj, K_trj, expected_cost_reduction)

    def forward_pass(self, learning_rate):
        """Forward pass of iLQR

        Args:
            learning_rate (double): learning rate (\alpha)
            
        Returns:
            ndarray: updated states, \hat x (8a,b,c)
            ndarray: updated inputs, \hat u (12)
        """
        # initialize before integration
        states = np.zeros((self.n_timesteps_ + 1, self.n_states_)) # to store updated trajectory
        inputs = np.zeros((self.n_timesteps_, self.n_inputs_)) # to store updated inputs
        current_state = self.init_state_

        # initialize and start integrating going forward for:
        # state = [., x_1, x_2, ..., x_N]
        # input =    [u_0, u_1, ..., u_{N-1}]
        states[1,:] = current_state # (8a), assume that index 0 contains x_0
        for i in range(0, self.n_timesteps_):
            # use current gains
            current_feedforward = learning_rate * self.k_feedforward_[i,:] # (12), \hat represents the updated variables
            current_feedback = self.K_feedback_[i,:,:] @ (current_state - self.states_[i,:]) # (12)
            current_input = self.inputs_[i,:] + current_feedforward + current_feedback
            # simulate
            next_state = self.f_(current_state, current_input, self.dt_, self.parameters_).flatten()
            # store states and inputs
            states[i+1,:] = next_state
            inputs[i,:] = current_input.flatten()
            
            # update states
            current_state = next_state

        return (states, inputs)

    def solve(self):
        """Solve all iLQR problem

        Returns:
            ndarray: Solved states
            ndarray: Solved inputs
            ndarray: Solved feedforward gains
            ndarray: Solved feedback gains
            double: final cost
        """
        # rollout with the initial guess
        [states, inputs] = self.rollout()

        # compute initial cost
        current_cost = self.compute_cost(states, inputs)

        # learning parameters
        learning_speed = 0.95 # this can be modified, 0.95 is very slow
        low_learning_rate = 0.05 # if learning rate drops to this value stop the optimization
        low_expected_reduction = 1e-3 # determines optimality
        
        armijo_threshold = 0.1 # determines if current line search solve is good (labelled as 'c' in (13))

        # start solving
        for i in range(0, self.n_iterations_):
            print("Starting iteration: {}".format(i))
            # backward pass
            (k_feedforward, K_feedback, expected_reduction) = self.backward_pass()

            # check for convergence
            if (np.abs(expected_reduction) < low_expected_reduction):
                # there is no further reduction, end the optimization
                print("Stopping optimization and accepting solution, abs_expected_reduction = {}".format(np.abs(expected_reduction)))
                break
            
            # start forward pass and line search with \alpha = 1
            learning_rate = 1
            armijo_flag = False
            # execute line search until the armijo condition is met (for now just check if the cost decreased)
            # TODO: add real armijo condition
            while (learning_rate > low_learning_rate and armijo_flag == False):
                # compute forward pass
                (new_states, new_inputs) = self.forward_pass(learning_rate)
                new_cost = self.compute_cost(new_states, new_inputs)

                # compute armijo condition
                cost_difference = (current_cost - new_cost)
                expected_cost_red = learning_rate * (self.expected_cost_reduction_grad_ + learning_rate*self.expected_cost_reduction_hess_)
                armijo_flag = (cost_difference / expected_cost_red) > armijo_threshold # z (13)

                print("Rate = {:.5f}, cost = {:.5f}, dcost = {:.5f}, expected cost red = {:.5f}, armijo = {}".format(learning_rate, new_cost, cost_difference, expected_cost_red, armijo_flag))

                if (armijo_flag):
                    # accept new trajectory if armijo condition is met
                    current_cost = new_cost
                    self.states_ = new_states
                    self.inputs_ = new_inputs
                else:
                    # no improvement, decrease learning rate and restart forward pass
                    learning_rate = learning_speed*learning_rate

            # if we exited while loop due to (learning_rate > low_learning_rate) being false
            if (learning_rate < low_learning_rate):
                print("Stopping optimization due to low learning rate")
                break

        # return the current trajectory
        states = self.states_
        inputs = self.inputs_

        return states, inputs, k_feedforward, K_feedback, current_cost


In [27]:
# import dynamics
(f,f_x,f_u) = symbolic_dynamics_pendulum()

# initialize timing
dt = 0.005
start_time = 0
end_time = 2
time_span = np.arange(start_time, end_time, dt)

# set states (start and end states are at rest)
n_states = 2 # position and velocity
n_inputs = 1 # inputs to the system
init_state = np.array([0.1,0])
target_state = np.array([np.pi,0])

# initial guess
initial_input_guess = 0.1*np.ones((time_span.shape[0], n_inputs))

# define weights
Q_k = np.zeros((n_states, n_states)) # just find a valid trajectory first
R_k = 0.001*np.eye(n_inputs)
Q_T = 100*np.eye(n_states)

# physical parameters
mass = 1
gravity = 9.8
pendulum_length = 1
parameters = np.array([mass, gravity, pendulum_length])

# iterations
n_iterations = 50

# ilqr
ilqr = ilqr(init_state, target_state, initial_input_guess, dt, start_time, end_time, f, f_x, f_u, Q_k, R_k, Q_T, parameters, n_iterations)

# solve for swing up
# (states, inputs, k_feedforward, K_feedback, current_cost) = ilqr.solve()

%time (states, inputs) = ilqr.rollout()
# print(ilqr.compute_cost(states, inputs))
# %time (k_trj, K_trj, expected_cost_reduction) = ilqr.backward_pass()


400
i = 0, current_input = [0.1], current_state = [0.1 0. ]
i = 1, current_input = [0.1], current_state = [ 0.1        -0.00439184]
i = 2, current_input = [0.1], current_state = [ 0.09997804 -0.00878367]
i = 3, current_input = [0.1], current_state = [ 0.09993412 -0.01317444]
i = 4, current_input = [0.1], current_state = [ 0.09986825 -0.01756307]
i = 5, current_input = [0.1], current_state = [ 0.09978043 -0.02194848]
i = 6, current_input = [0.1], current_state = [ 0.09967069 -0.02632961]
i = 7, current_input = [0.1], current_state = [ 0.09953904 -0.0307054 ]
i = 8, current_input = [0.1], current_state = [ 0.09938552 -0.03507476]
i = 9, current_input = [0.1], current_state = [ 0.09921014 -0.03943664]
i = 10, current_input = [0.1], current_state = [ 0.09901296 -0.04378996]
i = 11, current_input = [0.1], current_state = [ 0.09879401 -0.04813367]
i = 12, current_input = [0.1], current_state = [ 0.09855334 -0.05246671]
i = 13, current_input = [0.1], current_state = [ 0.09829101 -0.05678801]


In [28]:
states

array([[ 0.1       ,  0.        ],
       [ 0.1       , -0.00439184],
       [ 0.09997804, -0.00878367],
       [ 0.09993412, -0.01317444],
       [ 0.09986825, -0.01756307],
       [ 0.09978043, -0.02194848],
       [ 0.09967069, -0.02632961],
       [ 0.09953904, -0.0307054 ],
       [ 0.09938552, -0.03507476],
       [ 0.09921014, -0.03943664],
       [ 0.09901296, -0.04378996],
       [ 0.09879401, -0.04813367],
       [ 0.09855334, -0.05246671],
       [ 0.09829101, -0.05678801],
       [ 0.09800707, -0.06109652],
       [ 0.09770159, -0.06539118],
       [ 0.09737463, -0.06967094],
       [ 0.09702628, -0.07393476],
       [ 0.0966566 , -0.0781816 ],
       [ 0.09626569, -0.0824104 ],
       [ 0.09585364, -0.08662013],
       [ 0.09542054, -0.09080977],
       [ 0.09496649, -0.09497829],
       [ 0.0944916 , -0.09912466],
       [ 0.09399598, -0.10324786],
       [ 0.09347974, -0.10734688],
       [ 0.092943  , -0.11142072],
       [ 0.0923859 , -0.11546837],
       [ 0.09180856,

In [29]:
# # import dynamics
# (f,f_x,f_u) = symbolic_dynamics_pendulum()

# # initialize timing
# dt = 0.005
# start_time = 0
# end_time = 5
# time_span = np.arange(start_time, end_time, dt)

# # set states (start and end states are at rest)
# n_states = 2 # position and velocity
# n_inputs = 1 # inputs to the system
# init_state = np.array([0,0])
# target_state = np.array([np.pi,0])

# # initial guess
# initial_input_guess = 0.1*np.ones((time_span.shape[0], n_inputs))

# # define weights
# Q_k = np.zeros((n_states, n_states)) # just find a valid trajectory first
# R_k = 0.001*np.eye(n_inputs)
# Q_T = 100*np.eye(n_states)

# # physical parameters
# mass = 1
# gravity = 9.8
# pendulum_length = 1
# parameters = np.array([mass, gravity, pendulum_length])

# # iterations
# n_iterations = 50

# # ilqr
# ilqr = ilqr(init_state, target_state, initial_input_guess, dt, start_time, end_time, f, f_x, f_u, Q_k, R_k, Q_T, parameters, n_iterations)

# # solve for swing up
# (states, inputs, k_feedforward, K_feedback, current_cost) = ilqr.solve()

In [30]:
%matplotlib inline

import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

# subsample theta every 5 samples
_DECIMATE = 5
states_subsample = states[1:-1:_DECIMATE,0].copy()

pendulum_length = parameters[2]
x_pos = pendulum_length*np.sin(states_subsample)
y_pos = -pendulum_length*np.cos(states_subsample)

fig, ax = plt.subplots()
ax.set_xlim((-1.5, 1.5))
ax.set_ylim((-1.5, 1.5))
ax.set_aspect('equal', 'box')
plt.close(fig)

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    return (line, time_text)

def animate(i):
    x = [0, x_pos[i]]
    y = [0, y_pos[i]]
    line.set_data(x, y)
    time_text.set_text(time_template % (i*_DECIMATE*dt))
    return (line, time_text)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(states_subsample), interval=25, blit=True)

HTML(anim.to_html5_video())

https://en.wikipedia.org/wiki/Backtracking_line_search
